<a href="https://colab.research.google.com/github/parsa-abbasi/Sentiment-Analysis/blob/master/SentiPers/BinaryClassifier/NN/GoogleColab/DataAugmentation/FullTranslate/CNN_KerasEmb_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hazm
!pip install stopwords_guilannlp

    100% |████████████████████████████████| 317kB 30.3MB/s 
    100% |████████████████████████████████| 235kB 36.1MB/s 
    100% |████████████████████████████████| 1.4MB 20.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


# Import Libraries

In [2]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Convolution1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras import optimizers
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
import codecs
from stopwords_guilannlp import stopwords_output
from hazm import *
import matplotlib.pyplot as plt

Using TensorFlow backend.
paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


# File uploader

In [88]:
from google.colab import files
uploaded = files.upload()

Saving x_test.csv to x_test (2).csv
Saving x_train.csv to x_train (2).csv
Saving y_test.csv to y_test (2).csv
Saving y_train.csv to y_train (2).csv



# Import Dataset

In [102]:
x_train = pd.Series.from_csv('x_train (2).csv', sep='\t')
x_test = pd.Series.from_csv('x_test (2).csv', sep='\t')
y_train = pd.Series.from_csv('y_train (2).csv', sep='\t', header=0)
y_test = pd.Series.from_csv('y_test (2).csv', sep='\t', header=0)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [0]:
x_train = x_train.iloc[1:, ]
x_test = x_test.iloc[1:, ]

In [104]:
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (11122,)
x_test shape:  (1854,)
y_train shape:  (11122,)
y_test shape:  (1854,)


In [0]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [106]:
# See the data number of each category 
from collections import Counter
cnt = Counter(y_train)
cnt = dict(cnt)
print(cnt)

{1: 3246, 2: 1976, 0: 4818, -1: 1026, -2: 56}


In [107]:
binary_y_train = []
binary_y_test = []
binary_x_train = []
binary_x_test = []
for i, y in enumerate(y_train):
  if y != 0:
    if y > 0:
      binary_y_train.append(1)
      binary_x_train.append(x_train[i])
    else:
      binary_y_train.append(0)
      binary_x_train.append(x_train[i])
  else:
    binary_y_train.append(0)
    binary_x_train.append(x_train[i])
      
for i, y in enumerate(y_test):
  if y != 0:
    if y > 0:
      binary_y_test.append(1)
      binary_x_test.append(x_test[i])
    else:
      binary_y_test.append(0)
      binary_x_test.append(x_test[i])
  else:
    binary_y_test.append(0)
    binary_x_test.append(x_test[i])
      

x_train = np.asarray(binary_x_train)
x_test = np.asarray(binary_x_test)
y_train = np.asarray(binary_y_train)
y_test = np.asarray(binary_y_test)
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (11122,)
x_test shape:  (1854,)
y_train shape:  (11122,)
y_test shape:  (1854,)


In [108]:
# See the data number of each category 
from collections import Counter
cnt = Counter(y_train)
cnt = dict(cnt)
print(cnt)

{1: 5222, 0: 5900}


# Vectorize

In [0]:
puncs = ['،', '.', ',', ':', ';', '"']
normalizer = Normalizer()


# turn a doc into clean tokens
def clean_doc(doc):
  doc = normalizer.normalize(doc) # Normalize document using Hazm Normalizer
  tokenized = word_tokenize(doc)  # Tokenize text
  tokens = []
  for t in tokenized:
    temp = t
    for p in puncs:
      temp = temp.replace(p, '')
    tokens.append(temp)
  # tokens = [w for w in tokens if not w in stop_set]    # Remove stop words
  tokens = [w for w in tokens if not len(w) <= 1]
  tokens = [w for w in tokens if not w.isdigit()]
  tokens = ' '.join(tokens)
  return tokens

In [0]:
train_docs = list()
for document in x_train:
    train_docs.append(clean_doc(document))

In [0]:

# create the tokenizer
tokenizer = Tokenizer()

# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)

# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)

# pad sequences
max_length = max([len(s.split()) for s in train_docs])
x_train_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


In [0]:
test_docs = list()
for document in x_test:
    test_docs.append(clean_doc(document))

# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

In [0]:
encoded_docs = tokenizer.texts_to_sequences(test_docs)
x_test_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


# CNN Model

In [114]:
inp = Input(shape=(max_length, ))
x = Embedding(vocab_size, 2000, input_length=max_length)(inp)
x = Conv1D(filters=64, kernel_size=4, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=64, kernel_size=8, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=64, kernel_size=16, activation='relu', padding='same')(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(2000, activation="sigmoid")(x)
x = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

batch_size = 64
epochs = 5

hist = model.fit(x_train_padded, y_train, batch_size=batch_size, epochs=epochs)

test_loss, test_acc = model.evaluate(x_test_padded, y_test, verbose=0)

print('Test Accuracy: %f' % (test_acc*100))

train_loss, train_acc = model.evaluate(x_train_padded, y_train, verbose=0)

print('Train Accuracy: %f' % (train_acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 297)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 297, 2000)         19450000  
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 297, 64)           512064    
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 148, 64)           0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 148, 64)           32832     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 74, 64)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 74, 64)            65600     
__________